In [1]:
import jsbsim
import numpy as np
import scipy.optimize
import pandas as pd

import matplotlib.pyplot as plt
from pathlib import Path
import time
from logger import Logger

In [2]:
def set_location_purdue_airport(fdm):
    # location, Purdue airport
    fdm['ic/terrain-elevation-ft'] = 584.5
    fdm['ic/lat-geod-deg'] = 40.4110213580397
    fdm['ic/long-gc-deg'] = -86.92756398413263
    fdm['ic/psi-true-deg'] = 280

In [3]:
def trim(aircraft, ic, design_vector, x0, verbose, **kwargs):
    root = Path('.').resolve()
    fdm = jsbsim.FGFDMExec(str(root)) # The path supplied to FGFDMExec is the location of the folders "aircraft", "engines" and "systems"
    fdm.set_debug_level(0)    
    fdm.load_model(aircraft)
    fdm.suspend_integration()
    
    # set location
    set_location_purdue_airport(fdm)
    
    fdm_props = fdm.get_property_catalog('')
    for key in design_vector + list(ic.keys()):
        if key not in fdm_props.keys():
            raise KeyError(key)
            
    # cost function for trimming
    def cost(xd):
        # set initial condition
        for var in ic.keys():
            fdm[var] = ic[var]

        # set design vector
        for i, var in enumerate(design_vector):
            fdm[var] = xd[i]
        
        # set initial conditions
        fdm.run_ic()
        
        # trim propulsion
        prop = fdm.get_propulsion()
        for i in range(prop.get_num_engines()):
            prop.init_running(i)

        # compute cost, force moment balance
        udot = fdm['accelerations/udot-ft_sec2']
        vdot = fdm['accelerations/vdot-ft_sec2']
        wdot = fdm['accelerations/wdot-ft_sec2']
        pdot = fdm['accelerations/pdot-rad_sec2']
        qdot = fdm['accelerations/qdot-rad_sec2']
        rdot = fdm['accelerations/rdot-rad_sec2']
        return udot**2 + vdot**2 + wdot**2 + pdot**2 + qdot**2 + rdot**2
    
    res = scipy.optimize.minimize(fun=cost, x0=x0, **kwargs)
    if verbose:
        print(res)
    for i, var in enumerate(design_vector):
        ic[var] = res['x'][i]
    
    return ic, fdm

In [4]:
def simulate(aircraft, op_0, op_list=None, op_times=None, tf=50, realtime=False):
    if op_list is None:
        op_list = []
    
    if op_times is None:
        op_times = []
        
    root = Path('.').resolve()
    fdm = jsbsim.FGFDMExec(str(root)) # The path supplied to FGFDMExec is the location of the folders "aircraft", "engines" and "systems"

    # load model
    fdm.load_model(aircraft)
    fdm.set_output_directive(str(root/ 'data_output' / 'flightgear.xml'))
    fdm_props = fdm.get_property_catalog('')
    
    # add a method to check that properties being set exist
    def set_opereating_point(props):
        for key in props.keys():
            if key not in fdm_props.keys():
                raise KeyError(key)
            else:
                fdm[key] = props[key]
    
    # set location
    set_location_purdue_airport(fdm)
    
    # set operating points
    set_opereating_point(op_0)
    
    # start engines
    prop = fdm.get_propulsion()
    fdm.run_ic()
    for i in range(prop.get_num_engines()):
        prop.init_running(i)

    # start loop
    log = Logger()
    nice = True
    sleep_nseconds = 500
    initial_seconds = time.time()
    frame_duration = fdm.get_delta_t()
    op_count = 0
    result = fdm.run()

    while result and fdm.get_sim_time() < tf:
        t = fdm.get_sim_time()
        if op_count < len(op_times) and t > op_times[op_count]:
            set_opereating_point(op_list[op_count])
            op_count += 1

        if realtime:
            current_seconds = time.time()
            actual_elapsed_time = current_seconds - initial_seconds
            sim_lag_time = actual_elapsed_time - fdm.get_sim_time()

            for _ in range(int(sim_lag_time / frame_duration)):
                result = fdm.run()
                current_seconds = time.time()
        else:
            result = fdm.run()
            if nice:
                time.sleep(sleep_nseconds / 1000000.0)
        log.new_frame(
            t, fdm.get_property_catalog(''))

    log = log.to_pandas()
    return log

In [5]:
op_ground, fdm = trim(
    aircraft='F-35B-2',
    ic={
        'ic/vt-fps': 0,
        'gear/gear-cmd-norm': 1,
        'propulsion/engine/pitch-angle-rad': 0,
        'fcs/throttle-cmd-norm': 0,
        'fcs/throttle-cmd-norm': 0,
        'fcs/aileron-cmd-norm': 0,
        'fcs/elevator-cmd-norm': 0,
        'fcs/rudder-cmd-norm': 0,
        'fcs/left-brake-cmd-norm': 1,
        'fcs/right-brake-cmd-norm': 1,
        'fcs/center-brake-cmd-norm': 1,
    },
    design_vector=['ic/theta-rad', 'ic/h-agl-ft'],
    x0=[0, 0],
    verbose=True,
    method='Nelder-Mead', # works better with ground interaction
    tol=1e-12
)
op_ground

 final_simplex: (array([[4.85765918e-04, 3.70162834e+00],
       [4.85765918e-04, 3.70162834e+00],
       [4.85765918e-04, 3.70162834e+00]]), array([9.03386380e-12, 9.03386381e-12, 9.03386381e-12]))
           fun: 9.033863801684596e-12
       message: 'Optimization terminated successfully.'
          nfev: 300
           nit: 152
        status: 0
       success: True
             x: array([4.85765918e-04, 3.70162834e+00])


{'ic/vt-fps': 0,
 'gear/gear-cmd-norm': 1,
 'propulsion/engine/pitch-angle-rad': 0,
 'fcs/throttle-cmd-norm': 0,
 'fcs/aileron-cmd-norm': 0,
 'fcs/elevator-cmd-norm': 0,
 'fcs/rudder-cmd-norm': 0,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': 0.0004857659179286267,
 'ic/h-agl-ft': 3.7016283364266105}

In [6]:
log_ground = simulate(
    aircraft='F-35B-2',
    op_0=op_ground,
    tf=5,
    realtime=True)

In [7]:
op_hover, fdm = trim(
    aircraft='F-35B-2',
    ic={
        'ic/h-agl-ft': 30,
        'ic/vd-fps': 0,
        'ic/vn-fps': 0*np.cos(np.deg2rad(280)),
        'ic/ve-fps': 0*np.sin(np.deg2rad(280)),
        'ic/theta-rad': 0,
        'gear/gear-cmd-norm': 1,
        'fcs/left-brake-cmd-norm': 0,
        'fcs/right-brake-cmd-norm': 0,
        'fcs/center-brake-cmd-norm': 0,
    },
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
        'propulsion/engine/pitch-angle-rad',
    ],
    x0=[0, 0, np.deg2rad(90)],
    verbose=True,
    bounds=[[0, 2], [-1, 1], [np.deg2rad(0), np.deg2rad(120)]],
    tol=1e-30
)
op_hover

      fun: 7.95502465225636e-12
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([6.47673147e-10, 6.93364681e-12, 1.77555762e-11])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 280
      nit: 21
     njev: 70
   status: 2
  success: False
        x: array([ 1.01011378, -0.20000589,  1.57079635])


{'ic/h-agl-ft': 30,
 'ic/vd-fps': 0,
 'ic/vn-fps': 0.0,
 'ic/ve-fps': -0.0,
 'ic/theta-rad': 0,
 'gear/gear-cmd-norm': 1,
 'fcs/left-brake-cmd-norm': 0,
 'fcs/right-brake-cmd-norm': 0,
 'fcs/center-brake-cmd-norm': 0,
 'fcs/throttle-cmd-norm': 1.010113780038632,
 'fcs/elevator-cmd-norm': -0.20000589241972808,
 'propulsion/engine/pitch-angle-rad': 1.5707963510952738}

In [8]:
log_hover = simulate(
    aircraft='F-35B-2',
    op_0=op_hover,
    tf=5,
    realtime=True)

In [9]:
op_cruise, fdm = trim(
    aircraft='F-35B-2',
    ic={
        'ic/gamma-rad': 0,
        'ic/vt-fps': 800,
        'ic/h-agl-ft': 500,
        'gear/gear-cmd-norm': 0,
        'propulsion/engine/pitch-angle-rad': 0,
        'fcs/left-brake-cmd-norm': 0,
        'fcs/right-brake-cmd-norm': 0,
        'fcs/center-brake-cmd-norm': 0,
    },
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
        'fcs/rudder-cmd-norm',
        'fcs/aileron-cmd-norm',
        'ic/alpha-rad',
        'ic/beta-rad',
    ],
    x0=[0.5, 0, 0, 0, 0, 0],
    verbose=True,
    bounds=[[0, 2], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]],
)
op_cruise

      fun: 6.242881439632952e-10
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.11129733e-05, -3.10438777e-05,  3.04533405e-05,  1.39166369e-05,
        8.36139909e-04, -5.24394705e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 483
      nit: 54
     njev: 69
   status: 0
  success: True
        x: array([ 9.28218743e-01, -6.84827056e-02,  3.21122595e-04,  4.15289878e-06,
       -2.51992125e-03,  2.71617747e-04])


{'ic/gamma-rad': 0,
 'ic/vt-fps': 800,
 'ic/h-agl-ft': 500,
 'gear/gear-cmd-norm': 0,
 'propulsion/engine/pitch-angle-rad': 0,
 'fcs/left-brake-cmd-norm': 0,
 'fcs/right-brake-cmd-norm': 0,
 'fcs/center-brake-cmd-norm': 0,
 'fcs/throttle-cmd-norm': 0.9282187425141079,
 'fcs/elevator-cmd-norm': -0.06848270564465889,
 'fcs/rudder-cmd-norm': 0.0003211225945809392,
 'fcs/aileron-cmd-norm': 4.152898779530449e-06,
 'ic/alpha-rad': -0.0025199212482656767,
 'ic/beta-rad': 0.0002716177468953607}

In [10]:
log_cruise = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise,
    tf=5,
    realtime=True)